In [1]:
import cv2
import matplotlib
matplotlib.use('qt5agg')
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import ipywidgets as widgets
from IPython.display import display
cap = cv2.VideoCapture(0)

In [2]:
def ApSy(width,ant=50,freq=1e9,myres=10):
    AA=np.loadtxt("almapos.txt")
    almal_x=AA[0,:ant]
    almal_y=AA[1,:ant]
    wavel=3e8/freq
    bmax=wavel*width/2
    # Antenna position adjustment to fit max. baseline
    dx=almal_x-almal_x.min()
    dy=almal_y-almal_y.min()
    dmax=np.max([dx.max(),dy.max()])
    dnx=dx/dmax
    dny=dy/dmax
    x=bmax*dnx
    y=bmax*dny
    lx=np.zeros((ant,ant))
    ly=np.zeros((ant,ant))
    # calculate relative positions between antennae (baselines for the visibilities)
    for i in range(ant):
        for j in range(ant):
            lx[i,j]=(x[i]-x[j])
            ly[i,j]=(y[i]-y[j])
    # remember that x and y are swapped in imshow
    u=ly/wavel      # u values in wavenumber
    v=lx/wavel      # v values in wavenumber
    re_u=np.reshape(u,(len(x)**2),order='F') #reshape u into linear form
    re_v=np.reshape(v,(len(y)**2),order='F') 
    ore_u=re_u/myres
    ore_v=re_v/myres
    # UV gridding
    # create an empty matrix same size as in image for multiplying with fft of true sky image
    # this acts as a mask in fourier space (resolution of exactly 1 px)
    obs_uv_matrix=np.zeros((width,width))          
    for k in range(len(ore_u)): 
        int_u=int((ore_u)[k])
        int_v=int((ore_v)[k])
        obs_uv_matrix[int_u,int_v]=1.0
    obs_uv_matrix[0,0]=0.0 
    return obs_uv_matrix
    
def CreateImage(in_img,obs_uv_matrix):    
    
    fft_img=np.fft.fft2(in_img)              #This is the complex visibility from a complete and ideal array from u,v =0,0 outward.
    observed=fft_img*obs_uv_matrix           #Observed complex visibility. Combines ideal visibility with the observed uv points, removing those not seen by the selected array configuration.
    ifft_img=np.fft.ifft2(observed)          #inverse FFTs the observed visibility
    real_ifft_img=ifft_img.real              #Takes real part only for plotting
    return real_ifft_img

In [3]:
ret, frame = cap.read()
mns=np.min((frame.shape)[:2])

In [4]:
w = widgets.interactive(ApSy,width=widgets.fixed(mns), ant=(2,50), myres=(1,20),freq=widgets.fixed(1e9))

In [5]:
display(w)

In [14]:
fig = plt.figure(figsize=(10,5)) 
ax = fig.add_subplot(121)
ax2 = fig.add_subplot(122)
frame=frame[:mns,:mns]
im = ax.imshow(frame[:,:,0], animated=True,cmap='gray')
im2 = ax2.imshow(frame[:,:,0], animated=True,cmap='gray')

def updatefig(*args):
    ret, frame = cap.read()
    obs_uv_matrix=w.result
    frame=frame[:mns,:mns]
    newf=CreateImage(frame[:,:,0],obs_uv_matrix)
    im.set_array(newf)
    im2.set_array(frame[:,:,0])
    return im,im2
ani = animation.FuncAnimation(fig, updatefig, interval=10)
plt.show()